In [1]:
import csv
import warnings
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
import numpy as np
import json

In [2]:
pd.options.display.max_rows = 999
pd.options.display.max_columns=999
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [3]:
arquivoSAPBW = 'C:\\Temp\\ZTMM004_MURABEI\\OP_DAMM23.20221021.153854'

dfZTMM004_SAPBW = pd.read_csv(arquivoSAPBW, sep=';'
                              ,usecols=[0,1,2,5,7,12,15,16,17]
                              ,header=None 
                              #,skiprows=1
                              ,names=['CodContaCorrente'
                                      ,'TpoContaCorrente'
                                      ,'DtLanc'
                                      ,'CodCliente'
                                      ,'CodMaterial'
                                      ,'Qtd'
                                      ,'CodEmpresa'
                                      ,'CodCentro'
                                      ,'CodDeposito']
                                 ,dtype={'CodContaCorrente':'str'
                                      ,'TpoContaCorrente':'str'
                                      ,'DtLanc':'str'
                                      ,'CodCliente':'str'
                                      ,'CodMaterial':'str'
                                      ,'Qtd':'str'
                                      ,'CodEmpresa':'str'
                                      ,'CodCentro':'str'
                                      ,'CodDeposito':'str'}
                             ,date_parser=[2])

dfZTMM004_SAPBW['CodCliente'].fillna('0',inplace=True)

dfZTMM004_SAPBW['CodCliente'] = dfZTMM004_SAPBW['CodCliente'].astype(str).apply(lambda x: x.replace('A','').replace('C',''))
dfZTMM004_SAPBW['CodCliente'] = dfZTMM004_SAPBW['CodCliente'].astype(str).str.zfill(10)
#dfZTMM004_SAPBW.drop(dfZTMM004_SAPBW.loc[dfZTMM004_SAPBW['CodCliente']=='0000000000'].index, inplace=True)

dfZTMM004_SAPBW['Chave'] = dfZTMM004_SAPBW['CodContaCorrente'] \
                                + '_' + dfZTMM004_SAPBW['CodMaterial'] \
                                + '_' + dfZTMM004_SAPBW['CodCentro'] \
                                + '_' + dfZTMM004_SAPBW['CodCliente'] \
                                + '_' + dfZTMM004_SAPBW['CodDeposito'] #\
                                #+ '_' + dfZTMM004_SAPBW['DtLanc']

dfZTMM004_SAPBW['Valida'] = (dfZTMM004_SAPBW['Qtd'].astype(str).str.contains('-').replace('-','')*-1)

dfZTMM004_SAPBW.loc[dfZTMM004_SAPBW['Valida'] == '-1','Valida'] = -1
dfZTMM004_SAPBW.loc[dfZTMM004_SAPBW['Valida'] == 0,'Valida'] = 1

dfZTMM004_SAPBW['Qtd'] = (dfZTMM004_SAPBW['Qtd'].astype(str).str.replace('-','')).astype(float) * (dfZTMM004_SAPBW['Valida']).astype(float)

dfZTMM004_SAPBW.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4223 entries, 0 to 4222
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CodContaCorrente  4223 non-null   object 
 1   TpoContaCorrente  4223 non-null   object 
 2   DtLanc            4223 non-null   object 
 3   CodCliente        4223 non-null   object 
 4   CodMaterial       4223 non-null   object 
 5   Qtd               4223 non-null   float64
 6   CodEmpresa        4223 non-null   object 
 7   CodCentro         4223 non-null   object 
 8   CodDeposito       4216 non-null   object 
 9   Chave             4216 non-null   object 
 10  Valida            4223 non-null   int32  
dtypes: float64(1), int32(1), object(9)
memory usage: 346.5+ KB


In [4]:
resultZTMM004_SAPBW = dfZTMM004_SAPBW.groupby(['CodContaCorrente'
                                                 ,'CodCliente'
                                                 ,'CodMaterial'
                                                 ,'CodCentro'
                                                 ,'CodDeposito'
                                                 ,'Chave'],as_index=False).agg(QtdSAPBW=('Qtd','sum'))

In [5]:
resultZTMM004_SAPBW[resultZTMM004_SAPBW['Chave']=='006_000000000000000025_1061_0000001061_DISP']

,CodContaCorrente,CodCliente,CodMaterial,CodCentro,CodDeposito,Chave,QtdSAPBW
173,006,0000001061,000000000000000025,1061,DISP,006_000000000000000025_1061_0000001061_DISP,-14.0


In [6]:
arquivoSAPECC = 'C:\\Temp\\ZTMM004_MURABEI\\exportZTMM004_20221031_FULL.XLSX'

dfZTMM004_SAPECC = pd.read_excel(arquivoSAPECC
                                 ,usecols=[0,1,2,5,7,9,11,12]
                                 ,names=['CodContaCorrente'
                                      ,'TpoContaCorrente'
                                      ,'DtLanc'   
                                      ,'CodCliente'
                                      ,'CodMaterial'
                                      ,'Qtd'
                                      ,'CodCentro'
                                      ,'CodDeposito']
                                 ,dtype={'CodContaCorrente':'str'
                                      ,'TpoContaCorrente':'str'
                                      ,'DtLanc':'str'
                                      ,'CodCliente':'str'
                                      ,'CodMaterial':'str'
                                      ,'Qtd':'float'
                                      ,'CodCentro':'str'
                                      ,'CodDeposito':'str'}
                                )

dfZTMM004_SAPECC['CodContaCorrente'] = dfZTMM004_SAPECC['CodContaCorrente'].astype(str).str.zfill(3)

dfZTMM004_SAPECC['CodCliente'].fillna('0',inplace=True)
dfZTMM004_SAPECC['CodCliente'] = dfZTMM004_SAPECC['CodCliente'].astype(str).apply(lambda x: x.replace('A','').replace('C',''))
dfZTMM004_SAPECC['CodCliente'] = dfZTMM004_SAPECC['CodCliente'].astype(str).str.zfill(10)
#dfZTMM004_SAPECC.drop(dfZTMM004_SAPECC.loc[dfZTMM004_SAPECC['CodCliente']=='0000000000'].index, inplace=True)

dfZTMM004_SAPECC['Chave'] = dfZTMM004_SAPECC['CodContaCorrente'] \
                                + '_' + dfZTMM004_SAPECC['CodMaterial'] \
                                + '_' + dfZTMM004_SAPECC['CodCentro'] \
                                + '_' + dfZTMM004_SAPECC['CodCliente'] \
                                + '_' + dfZTMM004_SAPECC['CodDeposito']

dfZTMM004_SAPECC


,CodContaCorrente,TpoContaCorrente,DtLanc,CodCliente,CodMaterial,Qtd,CodCentro,CodDeposito,Chave
0,003,COMODATO,2022-01-03 00:00:00,0003894458,000000000000000025,-4.0,1027,DISP,003_000000000000000025_1027_0003894458_DISP
1,003,COMODATO,2022-01-03 00:00:00,0003881709,000000000000000024,-40.0,1019,DISP,003_000000000000000024_1019_0003881709_DISP
2,003,COMODATO,2022-01-03 00:00:00,0003817453,000000000000005046,-8.0,1065,DISP,003_000000000000005046_1065_0003817453_DISP
3,003,COMODATO,2022-01-04 00:00:00,0003857118,000000000000000025,-7.0,1019,DISP,003_000000000000000025_1019_0003857118_DISP
4,003,COMODATO,2022-01-04 00:00:00,0003554194,000000000000005046,-8.0,1185,ALMX,003_000000000000005046_1185_0003554194_ALMX
...,...,...,...,...,...,...,...,...,...
4218,024,JURÍDICO,2022-01-14 00:00:00,0001044695,000000000000000026,8.0,1132,NaN,NaN
4219,024,JURÍDICO,2022-01-17 00:00:00,0003728675,000000000000000024,265.0,1034,DISP,024_000000000000000024_1034_0003728675_DISP
4220,024,JURÍDICO,2022-01-21 00:00:00,0003805631,000000000000000024,-70.0,1143,DISP,024_000000000000000024_1143_0003805631_DISP
4221,024,JURÍDICO,2022-01-21 00:00:00,0003805631,000000000000000025,-5.0,1143,DISP,024_000000000000000025_1143_0003805631_DISP


In [7]:
resultZTMM004_SAPECC = dfZTMM004_SAPECC.groupby(['CodContaCorrente'
                                                 ,'CodCliente'
                                                 ,'CodMaterial'
                                                 ,'CodCentro'
                                                 ,'CodDeposito'
                                                 ,'Chave'],as_index=False).agg(QtdSAPECC=('Qtd','sum'))

In [8]:

resultZTMM004_SAPECC[resultZTMM004_SAPECC['Chave']=='006_000000000000000025_1061_0000001061_DISP']


,CodContaCorrente,CodCliente,CodMaterial,CodCentro,CodDeposito,Chave,QtdSAPECC
173,006,0000001061,000000000000000025,1061,DISP,006_000000000000000025_1061_0000001061_DISP,-14.0


In [19]:
resultZTMM004_FULL_SAPBW_TO_SAPECC = pd.merge(left=resultZTMM004_SAPBW
                              , right=resultZTMM004_SAPECC
                              , left_on='Chave'
                              , right_on='Chave'
                              , how='left'
                              )

resultZTMM004_FULL_SAPBW_TO_SAPECC.rename(columns={'CodContaCorrente_x':'CodContaCorrente'
                                   ,'CodMaterial_x':'CodMaterial'
                                   ,'CodCentro_x':'CodCentro'
                                   ,'CodCliente_x':'CodCliente'},inplace=True)

# ETAPA DE COMPARAÇÃO COM OS DADOS DO SAP BW E OS DADOS DO SAP ECC - Coluna Quantidade
resultZTMM004_FULL_SAPBW_TO_SAPECC.loc[resultZTMM004_FULL_SAPBW_TO_SAPECC['QtdSAPBW'] == resultZTMM004_FULL_SAPBW_TO_SAPECC['QtdSAPECC'], 'Comparacao_Qtd'] = 'Igual'
resultZTMM004_FULL_SAPBW_TO_SAPECC.loc[resultZTMM004_FULL_SAPBW_TO_SAPECC['QtdSAPBW'] != resultZTMM004_FULL_SAPBW_TO_SAPECC['QtdSAPECC'], 'Comparacao_Qtd'] = 'Diferente'

resultComparacao_ZTMM004_FULL_SAPBW_TO_SAPECC = resultZTMM004_FULL_SAPBW_TO_SAPECC[['Chave'
                                                    ,'CodContaCorrente'
                                                    ,'CodMaterial'
                                                    ,'CodCentro'
                                                    ,'CodCliente'
                                                    ,'QtdSAPBW'
                                                    ,'QtdSAPECC'
                                                    ,'Comparacao_Qtd'
                                                   ]]

resultComparacao_ZTMM004_FULL_SAPBW_TO_SAPECC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 793 entries, 0 to 792
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Chave             793 non-null    object 
 1   CodContaCorrente  793 non-null    object 
 2   CodMaterial       793 non-null    object 
 3   CodCentro         793 non-null    object 
 4   CodCliente        793 non-null    object 
 5   QtdSAPBW          793 non-null    float64
 6   QtdSAPECC         793 non-null    float64
 7   Comparacao_Qtd    793 non-null    object 
dtypes: float64(2), object(6)
memory usage: 55.8+ KB


In [20]:
resultComparacao_ZTMM004_FULL_SAPBW_TO_SAPECC

,Chave,CodContaCorrente,CodMaterial,CodCentro,CodCliente,QtdSAPBW,QtdSAPECC,Comparacao_Qtd
0,003_000000000000005046_1151_0000012026_DISP,003,000000000000005046,1151,0000012026,4.0,4.0,Igual
1,003_000000000000005046_1051_0000017371_DISP,003,000000000000005046,1051,0000017371,4.0,4.0,Igual
2,003_000000000000000026_1051_0000017452_DISP,003,000000000000000026,1051,0000017452,4.0,4.0,Igual
3,003_000000000000000024_1045_0000023193_DISP,003,000000000000000024,1045,0000023193,0.0,0.0,Igual
4,003_000000000000000024_1073_0000190172_DISP,003,000000000000000024,1073,0000190172,1150.0,1150.0,Igual
5,003_000000000000000026_1171_0000903568_DISP,003,000000000000000026,1171,0000903568,-2.0,-2.0,Igual
6,003_000000000000000026_1041_0000916893_DISP,003,000000000000000026,1041,0000916893,2.0,2.0,Igual
7,003_000000000000000024_1013_0000942688_DISP,003,000000000000000024,1013,0000942688,40.0,40.0,Igual
8,003_000000000000000025_1019_0000945895_DISP,003,000000000000000025,1019,0000945895,-6.0,-6.0,Igual
9,003_000000000000000024_1051_0000952197_DISP,003,000000000000000024,1051,0000952197,800.0,800.0,Igual


In [17]:
resultZTMM004_FULL_SAPECC_TO_SAPBW = pd.merge(left=resultZTMM004_SAPECC
                              , right=resultZTMM004_SAPBW
                              , left_on='Chave'
                              , right_on='Chave'
                              , how='left'
                              )

resultZTMM004_FULL_SAPECC_TO_SAPBW.rename(columns={'CodContaCorrente_x':'CodContaCorrente'
                                   ,'CodMaterial_x':'CodMaterial'
                                   ,'CodCentro_x':'CodCentro'
                                   ,'CodCliente_x':'CodCliente'},inplace=True)

# ETAPA DE COMPARAÇÃO COM OS DADOS DO SAP BW E OS DADOS DO SAP ECC - Coluna Quantidade
resultZTMM004_FULL_SAPECC_TO_SAPBW.loc[resultZTMM004_FULL_SAPECC_TO_SAPBW['QtdSAPBW'] == resultZTMM004_FULL_SAPECC_TO_SAPBW['QtdSAPECC'], 'Comparacao_Qtd'] = 'Igual'
resultZTMM004_FULL_SAPECC_TO_SAPBW.loc[resultZTMM004_FULL_SAPECC_TO_SAPBW['QtdSAPBW'] != resultZTMM004_FULL_SAPECC_TO_SAPBW['QtdSAPECC'], 'Comparacao_Qtd'] = 'Diferente'

resultComparacao_ZTMM004_FULL_SAPECC_TO_SAPBW = resultZTMM004_FULL_SAPECC_TO_SAPBW[['Chave'
                                                    ,'CodContaCorrente'
                                                    ,'CodMaterial'
                                                    ,'CodCentro'
                                                    ,'CodCliente'
                                                    ,'QtdSAPBW'
                                                    ,'QtdSAPECC'
                                                    ,'Comparacao_Qtd'
                                                   ]]

resultComparacao_ZTMM004_FULL_SAPECC_TO_SAPBW.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 793 entries, 0 to 792
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Chave             793 non-null    object 
 1   CodContaCorrente  793 non-null    object 
 2   CodMaterial       793 non-null    object 
 3   CodCentro         793 non-null    object 
 4   CodCliente        793 non-null    object 
 5   QtdSAPBW          793 non-null    float64
 6   QtdSAPECC         793 non-null    float64
 7   Comparacao_Qtd    793 non-null    object 
dtypes: float64(2), object(6)
memory usage: 55.8+ KB


In [18]:
resultComparacao_ZTMM004_FULL_SAPECC_TO_SAPBW

,Chave,CodContaCorrente,CodMaterial,CodCentro,CodCliente,QtdSAPBW,QtdSAPECC,Comparacao_Qtd
0,003_000000000000005046_1151_0000012026_DISP,003,000000000000005046,1151,0000012026,4.0,4.0,Igual
1,003_000000000000005046_1051_0000017371_DISP,003,000000000000005046,1051,0000017371,4.0,4.0,Igual
2,003_000000000000000026_1051_0000017452_DISP,003,000000000000000026,1051,0000017452,4.0,4.0,Igual
3,003_000000000000000024_1045_0000023193_DISP,003,000000000000000024,1045,0000023193,0.0,0.0,Igual
4,003_000000000000000024_1073_0000190172_DISP,003,000000000000000024,1073,0000190172,1150.0,1150.0,Igual
5,003_000000000000000026_1171_0000903568_DISP,003,000000000000000026,1171,0000903568,-2.0,-2.0,Igual
6,003_000000000000000026_1041_0000916893_DISP,003,000000000000000026,1041,0000916893,2.0,2.0,Igual
7,003_000000000000000024_1013_0000942688_DISP,003,000000000000000024,1013,0000942688,40.0,40.0,Igual
8,003_000000000000000025_1019_0000945895_DISP,003,000000000000000025,1019,0000945895,-6.0,-6.0,Igual
9,003_000000000000000024_1051_0000952197_DISP,003,000000000000000024,1051,0000952197,800.0,800.0,Igual


In [24]:
dfZTMM004_SAPBW[['CodContaCorrente'
              ,'TpoContaCorrente'
              ,'CodCliente'
              ,'CodMaterial'
              ,'CodEmpresa'
              ,'CodCentro'
              ,'CodDeposito'
              ,'DtLanc'
              ,'Qtd']].to_csv('ARQ_HML_TRANSACAO_ZTMM004_ARACARI.csv',sep=';',index=False)